In [177]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,ParameterGrid
import optuna.integration.lightgbm as lgb
from sklearn.model_selection import cross_val_score
import optuna
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import roc_auc_score
%matplotlib inline

feature_fraction, val_score: inf:   0%|          | 0/7 [03:31<?, ?it/s]


In [ ]:
train_df = pd.read_csv('application_train.csv')
test_df = pd.read_csv('application_test.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
bureau = pd.read_csv('bureau.csv')
credit_card_balance = pd.read_csv('credit_card_balance.csv')
HomeCredit_columns_description = pd.read_csv('HomeCredit_columns_description.csv')
installments_payments =  pd.read_csv('installments_payments.csv')
POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv')
previous_application = pd.read_csv('previous_application.csv')

In [ ]:
train_df.info()

In [ ]:
train_df.isnull().sum()

In [ ]:
def missing_value_table(df):
    #total missing value
    mis_val = df.isnull().sum()

    #percentage of missing value
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    #make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent],axis=1)

    #rename the columns 
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0 : 'missing values',1 : '% of total values'}
    )
    #sort the table by percentage of missing decsending
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of total values',ascending=False).round(1)

    #print summary information
    print('Your selected dataframe has ' + str(df.shape[1]) + ' columns.\n'
          'There are ' + str(mis_val_table_ren_columns.shape[0]) + 
          ' columns that have missing values'
    )

    #return the dataframe with missing information
    return mis_val_table_ren_columns


In [ ]:
missing_value_table(train_df)

In [ ]:
train_df.select_dtypes('object').apply(pd.Series.nunique,axis=0)

In [ ]:
le = LabelEncoder()
le_count = 0

for col in train_df:
    if train_df[col].dtype == 'object':
        if len(list(train_df[col].unique())) <= 2:

            le.fit(train_df[col])
            train_df[col] = le.transform(train_df[col])
            test_df[col] = le.transform(test_df[col])

            le_count += 1

print('%d columns were label encoded' %le_count)

In [ ]:
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)

print('Training Feature shape: ', train_df.shape)
print('Test Feature shape: ', test_df.shape)

In [ ]:
train_labels = train_df['TARGET']

train_df, test_df = train_df.align(test_df, join='inner',axis=1)

train_df['TARGET'] = train_labels

print('Training Feature Shape: ', train_df.shape)
print('Test Feature Shape: ', test_df.shape)

In [ ]:
(train_df['DAYS_BIRTH'] / -365).describe()

In [ ]:
train_df['DAYS_EMPLOYED'].describe()

In [ ]:
train_df['DAYS_EMPLOYED'].plot.hist(title='Days Employed Histogram')
plt.xlabel('Days Employment')

In [ ]:
#create an anomalous flag column
train_df['DAYS_EMPLOYED_ANOM'] = train_df['DAYS_EMPLOYED'] == 365243

#Replace the anomalous values with nan
train_df['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)

train_df['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram')
plt.xlabel('Days Employment')

In [ ]:
test_df['DAYS_EMPLOYED_ANOM'] = test_df['DAYS_EMPLOYED'] == 365243
test_df['DAYS_EMPLOYED'].replace({365243:np.nan},inplace=True)

print('There are %d anomalies in the test data out of %d entries' % (test_df['DAYS_EMPLOYED_ANOM'].sum(),len(test_df)))  

In [ ]:
#Find correlations with the target and sort
correlations = train_df.corr()['TARGET'].sort_values()

#Display correlations
print('Most Positive Correlations:\n',correlations.tail(15))
print('\nMost Negative Correlaitons:\n',correlations.head(15))


In [ ]:
#Find the correlation of the positive days since birth and target

train_df['DAYS_BIRTH'] = abs(train_df['DAYS_BIRTH'])
train_df['DAYS_BIRTH'].corr(train_df['TARGET'])


In [ ]:
#set the style of plots
plt.style.use('fivethirtyeight')

#plot the distribution of ages in years 
plt.hist(train_df['DAYS_BIRTH']/365, edgecolor='k', bins=25)
plt.title('Age of Client'); plt.xlabel('Age(years)'); plt.ylabel('Count'); 


In [ ]:
plt.figure(figsize=(12,8))

#KDE plot of loans that were repaid on time 
sns.kdeplot(train_df.loc[train_df['TARGET'] == 0, 'DAYS_BIRTH'] /365,label='target = 0')

#KDE plot of loans that were not repaid on time
sns.kdeplot(train_df.loc[train_df['TARGET'] == 1, 'DAYS_BIRTH']/365,label='target = 1')

# Labeling of plot
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');


In [ ]:
#Age information into a separate dataframe
age_data = train_df[['TARGET','DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH']/365

#Bin the age data
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'],bins=np.linspace(20,70,num=11))
age_data.head(10)

In [ ]:
#Group by the bin and calculate averages
age_groups = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize=(8,8))
#graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str),100*age_groups['TARGET'])

#plot labeling
plt.xticks(rotation=75)
plt.xlabel('Age Group(years)')
plt.ylabel('Failure to repay(%)')
plt.title('Failure to repay by age group')

In [ ]:
#Extract the EXT_SOURCE variables and show correlations
ext_data = train_df[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

In [ ]:
sns.heatmap(ext_data_corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)
plt.title('Correlation Heatmap');


In [ ]:
plt.figure(figsize=(10,12))

#iterate through the sources
for i, source in enumerate(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']):
    #create a new subplot for each sources
    plt.subplot(3,1,i+1)
    #plot repaid loans
    sns.kdeplot(train_df.loc[train_df['TARGET'] == 0, source], label = 'target=0')
    #plot loans that were not repaid
    sns.kdeplot(train_df.loc[train_df['TARGET'] == 1, source], label='target = 1')

    #label the plots
    plt.title('Distribution of %s by Target Value'%source)
    plt.xlabel('%s'%source)
    plt.ylabel('Density')


plt.tight_layout(h_pad = 2.5)


In [ ]:
#copy the data for plotting
plot_data = ext_data.drop(columns=['DAYS_BIRTH']).copy()

#Add in the age of client in years
plot_data['YEARS_BIRTH'] = age_data['YEARS_BIRTH']

#Drop na values and limit to first 100000 rows 
plot_data = plot_data.dropna().loc[:100000, :]

#Function to calculate correlation coefficient between two values
def corr_func(x, y, **kwargs):
    r = np.corrcoef(x, y)[0][1]
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.2, .8), xycoords=ax.transAxes,
                size = 20)

#Create the pairgrid object 
grid = sns.PairGrid(data = plot_data, height = 3, diag_sharey=False,
                    hue = 'TARGET', 
                    vars = [x for x in list(plot_data.columns) if x != 'TARGET'])

#upper is a scatter plot                
grid.map_upper(plt.scatter, alpha = 0.2)

# Bottom is density plot
grid.map_lower(sns.kdeplot, cmap = plt.cm.OrRd_r);

plt.suptitle('Ext Source and Age Features Pairs Plot', height = 32, y = 1.05);


In [ ]:
#Make a new dataframe for polynomial features
poly_features = train_df[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH','TARGET']]

poly_features_test = test_df[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]

#imputer for handling missing value
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

poly_target = poly_features['TARGET']

poly_features = poly_features.drop(columns=['TARGET'])

#Need to impute missing values
poly_features = imputer.fit_transform(poly_features)

poly_features_test = imputer.transform(poly_features_test)

from sklearn.preprocessing import PolynomialFeatures

#Create the polynomial object with specified degree
poly_transformer =  PolynomialFeatures(degree = 3)


In [ ]:
#train the polynomial feature 
poly_transformer.fit(poly_features)

#Transform the feature
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)

print('Polynomial Features Shape: ', poly_features.shape)


In [ ]:
poly_transformer.get_feature_names(input_features=['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH'])[:15]


In [ ]:
#Create a dataframe of the features
poly_features = pd.DataFrame(poly_features,
                             columns = poly_transformer.get_feature_names([
                                 'EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH'
                             ])
)

#Add in the target
poly_features['TARGET'] = poly_target

#Find the correlation with the target
poly_corrs = poly_features.corr()['TARGET'].sort_values()

#Display most negetive and most positive
print(poly_corrs.head(10))
print(poly_corrs.tail(5))


In [ ]:
# Put test features into dataframe
poly_features_test = pd.DataFrame(poly_features_test, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                                'EXT_SOURCE_3', 'DAYS_BIRTH']
))

#Merge polynomial feature into training dataframe
poly_features['SK_ID_CURR'] = train_df['SK_ID_CURR']
train_df_poly = train_df.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# Merge polynomial features into testing dataframe
poly_features_test['SK_ID_CURR'] = test_df['SK_ID_CURR']
test_df_poly = test_df.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

#Align the dataframe
train_df_poly,test_df_poly = train_df_poly.align(test_df_poly,join='inner',axis=1)

# Print out the new shapes
print('Training data with polynomial features shape: ', train_df_poly.shape)
print('Testing data with polynomial features shape:  ', test_df_poly.shape)


In [ ]:
train_df_domain = train_df.copy()
test_df_domain = test_df.copy()

train_df_domain['CREDIT_INCOME_PERCENT'] = train_df_domain['AMT_CREDIT'] / train_df_domain['AMT_INCOME_TOTAL']
train_df_domain['ANNUITY_INCOME_PERCENT'] = train_df_domain['AMT_ANNUITY'] / train_df_domain['AMT_INCOME_TOTAL']
train_df_domain['CREDIT_TERM'] = train_df_domain['AMT_ANNUITY'] / train_df_domain['AMT_CREDIT']
train_df_domain['DAYS_EMPLOYED_PERCENT'] = train_df_domain['DAYS_EMPLOYED'] / train_df_domain['DAYS_BIRTH']

In [ ]:
test_df_domain['CREDIT_INCOME_PERCENT'] = test_df_domain['AMT_CREDIT'] / test_df_domain['AMT_INCOME_TOTAL']
test_df_domain['ANNUITY_INCOME_PERCENT'] = test_df_domain['AMT_ANNUITY'] / test_df_domain['AMT_INCOME_TOTAL']
test_df_domain['CREDIT_TERM'] = test_df_domain['AMT_ANNUITY'] / test_df_domain['AMT_CREDIT']
test_df_domain['DAYS_EMPLOYED_PERCENT'] = test_df_domain['DAYS_EMPLOYED'] / test_df_domain['DAYS_BIRTH']

In [ ]:
train_df['TARGET']

In [ ]:
X = train_df_poly.drop(columns=['SK_ID_CURR'])
y = train_df['TARGET']
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [ ]:
import re
data = train_df_poly.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test_df_poly = test_df_poly.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
test_df_poly = test_df_poly.drop(columns=['SK_ID_CURR'])

In [ ]:
X = data.drop(columns=['SK_ID_CURR'])
y = train_df['TARGET']
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [ ]:
lgb_train = lgb.Dataset(X_train,y_train)
lgb_test = lgb.Dataset(X_test,y_test,reference=lgb_train)

#scores = cross_val_score(rfc,X_train,y_train)


In [178]:
# LightGBMのパラメータ設定
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'vervose': 0
}

In [179]:

# ベストなパラメータ、途中経過を保存する
params = {
    'objective': 'binary',
    'metric': 'auc',
    "verbosity": -1,
    "boosting_type": "gbdt",
}

best_params, history = {}, []

# LightGBM学習
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=[lgb_train, lgb_test],
                early_stopping_rounds=10
               )

best_params = gbm.params
best_params

[I 2022-01-02 15:06:48,614] A new study created in memory with name: no-name-fc18c23f-dfac-473a-b045-8caabecf6933
feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's auc: 0.720947	valid_1's auc: 0.718787
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc: 0.728289	valid_1's auc: 0.725479
[3]	valid_0's auc: 0.733115	valid_1's auc: 0.72949
[4]	valid_0's auc: 0.735357	valid_1's auc: 0.731046
[5]	valid_0's auc: 0.737582	valid_1's auc: 0.733251
[6]	valid_0's auc: 0.739344	valid_1's auc: 0.735058
[7]	valid_0's auc: 0.740533	valid_1's auc: 0.735558
[8]	valid_0's auc: 0.741733	valid_1's auc: 0.737203
[9]	valid_0's auc: 0.742908	valid_1's auc: 0.73802
[10]	valid_0's auc: 0.744422	valid_1's auc: 0.73919
[11]	valid_0's auc: 0.745508	valid_1's auc: 0.739907
[12]	valid_0's auc: 0.747022	valid_1's auc: 0.741038
[13]	valid_0's auc: 0.74813	valid_1's auc: 0.741801
[14]	valid_0's auc: 0.748941	valid_1's auc: 0.742241
[15]	valid_0's auc: 0.750071	valid_1's auc: 0.74271
[16]	valid_0's auc: 0.751349	valid_1's auc: 0.743543
[17]	valid_0's auc: 0.752145	valid_1's auc: 0.743945
[18]	valid_0's auc: 0.753259	valid_1's auc: 0.744799

feature_fraction, val_score: 0.756646:  14%|#4        | 1/7 [00:30<03:05, 30.93s/it]

[91]	valid_0's auc: 0.803537	valid_1's auc: 0.756561
[92]	valid_0's auc: 0.804024	valid_1's auc: 0.75657
Early stopping, best iteration is:
[82]	valid_0's auc: 0.798994	valid_1's auc: 0.756646


/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's auc: 0.722595	valid_1's auc: 0.719442
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc: 0.729066	valid_1's auc: 0.724673
[3]	valid_0's auc: 0.733756	valid_1's auc: 0.728634
[4]	valid_0's auc: 0.736115	valid_1's auc: 0.731167
[5]	valid_0's auc: 0.737846	valid_1's auc: 0.73326
[6]	valid_0's auc: 0.739695	valid_1's auc: 0.735149
[7]	valid_0's auc: 0.741139	valid_1's auc: 0.736032
[8]	valid_0's auc: 0.74233	valid_1's auc: 0.737156
[9]	valid_0's auc: 0.743365	valid_1's auc: 0.737782
[10]	valid_0's auc: 0.744124	valid_1's auc: 0.738356
[11]	valid_0's auc: 0.745066	valid_1's auc: 0.739078
[12]	valid_0's auc: 0.746087	valid_1's auc: 0.739698
[13]	valid_0's auc: 0.74721	valid_1's auc: 0.740586
[14]	valid_0's auc: 0.74775	valid_1's auc: 0.740919
[15]	valid_0's auc: 0.749008	valid_1's auc: 0.741701
[16]	valid_0's auc: 0.749495	valid_1's auc: 0.742011
[17]	valid_0's auc: 0.750408	valid_1's auc: 0.742619
[18]	valid_0's auc: 0.751312	valid_1's auc: 0.74334

min_data_in_leaf, val_score: 0.760142: 100%|##########| 5/5 [02:54<00:00, 34.87s/it]

[100]	valid_0's auc: 0.78803	valid_1's auc: 0.759919
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.78803	valid_1's auc: 0.759919


{'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 8.124133728185353,
 'lambda_l2': 9.263490933804805,
 'num_leaves': 31,
 'feature_fraction': 0.5,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 100,
 'early_stopping_round': 10}

In [186]:
predict1 = gbm.predict(test_df_poly)


In [189]:
p.describe()

,0
count,48744.000000
mean,0.082497
std,0.066557
min,0.008280
25%,0.036306
50%,0.061544
75%,0.106527
max,0.585964


In [ ]:
pred1 = lgbcls.predict(X_test)


In [192]:
prediction = np.where(predict1 < 0.08,0,1)
prediction

array([0, 1, 0, ..., 0, 0, 1])

In [ ]:
predict = lgbcls.predict(test_df_poly)
lgbcls.predict()

In [ ]:
submit = test_df[['SK_ID_CURR']]
submit['TARGET'] = predict
submit.to_csv('submit1.csv',index=False)

In [193]:
submit = test_df[['SK_ID_CURR']]
submit['TARGET'] = prediction
submit.to_csv('submit3.csv',index=False)

<ipython-input-193-3b1a1eef803e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['TARGET'] = prediction
